<a href="https://colab.research.google.com/github/aliyyah-u/NLP_Medical_NER/blob/main/NLP_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
__author__ = "Aliyyah U."
__version__ = "IN3045 City St George's, University of London, Spring 2025"
#Ideally, we would reuse trusted libraries to perform NLP. Some of the commonly recommended ones are: , nltk, spacy, torchtext, huggingface: transformers, datasets.

# CLEAN

In [ ]:
#load dataset
from datasets import load_dataset
!pip install -U huggingface_hub
from huggingface_hub import login
login()
dataset = load_dataset("parsa-mhmdi/Medical_NER")
print(dataset)

#see dataset samples
print(dataset["train"][0]["tokens"]) #can see that dataset is already tokenised
print(dataset["train"][0]["ner_tags"]) #can see that tags already in BIO format

#split dataset for testing
dataset = dataset['train'].train_test_split(test_size=0.1)
print(dataset)

#see contents of training dataset
ner_feature = dataset["train"].features
print(ner_feature)

#print sample of tokens with correct ner_tags
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

def match_tokens_labels(tokens, labels):
  line1 = ""
  line2 = ""
  for word, label in zip(words, labels):
    max_length = max(len(word), len(label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += label + " " * (max_length - len(label) + 1)
  print(line1)
  print(line2)

match_tokens_labels(words, labels)

#convert dataset contents into lists for further processing
df = dataset["train"].to_pandas()
train_tokens = df["tokens"].tolist()
train_tags = df["ner_tags"].tolist()

print("\nSample of training tokens:")
print(train_tokens[:2])
print("\nSample of training tags:")
print(train_tags[:2])

# See all unique tag names
train_unique_tags = set(tag for sublist in train_tags for tag in sublist)
print("\nAll unique NER tags in train dataset:")
print(sorted(train_unique_tags))

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import random

# Load a real BPE tokenizer (GPT-2 uses BPE)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Load dataset
dataset = load_dataset("parsa-mhmdi/Medical_NER")
dataset = dataset['train'].train_test_split(test_size=0.1)

# Build the list of unique NER tags from the dataset
ner_tags = list(set(tag for example in dataset["train"]["ner_tags"] for tag in example))
ner_tags.sort()  # Optional: sort for consistency
print(f"NER Tags: {ner_tags}")

# Function to tokenize using internal BPE merges (now using tokenizer methods)
def tokenize(text):
    # Tokenize the text with GPT-2 tokenizer
    bpe_tokens = tokenizer.tokenize(text)
    return bpe_tokens

# Assign random NER tags to BPE tokens
def generate_random_bpe_ner(tokens):
    text = " ".join(tokens)
    bpe_tokens = tokenize(text)
    random_tags = [random.choice(ner_tags) for _ in bpe_tokens]
    return bpe_tokens, random_tags

# Pretty-print function
def match_tokens_labels(tokens, labels):
    line1 = ""
    line2 = ""
    for word, label in zip(tokens, labels):
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    print(line1)
    print(line2)

# Example usage
mytokens = dataset["train"][0]["tokens"]
bpe_tokens, bpe_labels = generate_random_bpe_ner(mytokens)
match_tokens_labels(bpe_tokens, bpe_labels)

In [ ]:
from datasets import load_dataset
!pip install -U huggingface_hub
from huggingface_hub import login
login()

dataset = load_dataset("parsa-mhmdi/Medical_NER")

print(dataset)

print(dataset["train"][0]["tokens"])
print(dataset["train"][0]["ner_tags"])

dataset = dataset['train'].train_test_split(test_size=0.1)
print(dataset)

ner_feature = dataset["train"].features
print(ner_feature)

def match_tokens_labels(tokens, labels):
  line1 = ""
  line2 = ""
  for word, label in zip(words, labels):
    max_length = max(len(word), len(label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += label + " " * (max_length - len(label) + 1)
  print(line1)
  print(line2)

words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]
match_tokens_labels(words, labels)

# fyi - View all tags
train_unique_tags = set(tag for sublist in train_tags for tag in sublist)
print("\nAll unique training NER tags:")
print(sorted(train_unique_tags))

import random

# Function to match tokens with random NER labels
def match_tokens_labels(tokens, train_tags):
    # Create the list of unique tags
    all_unique_tags = sorted(set(tag for sublist in train_tags for tag in sublist))

    # Use random.choices() to assign random tags (with replacement) to each token
    random_tags = random.choices(all_unique_tags, k=len(tokens))  # Allow repeated sampling

    # Prepare strings for printing
    line1 = ""
    line2 = ""

    for word, label in zip(tokens, random_tags):
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)

    # Print the tokens and their randomly assigned labels
    print(line1)
    print(line2)

# Example usage (make sure you pass tokens and the train_tags from your dataset)
mytokens = dataset["train"][0]["tokens"]
train_tags = dataset["train"]["ner_tags"]

match_tokens_labels(mytokens, train_tags)

# MESSY


## load dataset


In [ ]:
!pip install --upgrade huggingface_hub

# Login to access dataset
from huggingface_hub import login
login()

import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_parquet("hf://datasets/parsa-mhmdi/Medical_NER/data/train-00000-of-00001.parquet")

train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=42)

train_data = df.loc[train_indices]
test_data = df.loc[test_indices]

# Preview samples from train and test, check df is correctly loaded
print("Train sample:")
print(train_data.head())
print("\nTest sample:")
print(test_data.head())

# Check data types of 'tokens' and 'ner_tags' columns
print("\nData type of train_data.tokens:")
print(type(train_data['tokens']))
print("\nData type of train_data.ner_tags:")
print(type(train_data['ner_tags']))

# Convert to list
train_tokens = train_data.tokens.tolist()
train_tags = train_data.ner_tags.tolist()
print("\nSample of training tokens:")
print(train_tokens[:5])
print("\nSample of training tags:")
print(train_tags[:5])

# fyi - View all tags
train_unique_tags = set(tag for sublist in train_tags for tag in sublist)
print("\nAll unique training NER tags:")
print(sorted(train_unique_tags))

## Part of Speech Tagging

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(df[tokens])

for token in doc:
  print(token, "|", token.pos_)

## lab

## Simple tokeniser

In [ ]:
!pip install -U huggingface_hub
from huggingface_hub import login
login()
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_parquet("hf://datasets/parsa-mhmdi/Medical_NER/data/train-00000-of-00001.parquet")
print('\n',df.info())

#split for test dataset
train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=42)
train_data = df.loc[train_indices]
test_data = df.loc[test_indices]

print("\nTrain token sample:")
print(train_data.tokens[:5]) #dataset is already tokenised on whitespace/capital letters, & normalised by lowercasing

## Stemming

In [ ]:
def simple_stemmer(token):
    # we will only work over words > 2 characters.
    if len(token) <= 2:
        return token

    suffixes = ['al', 'ance', 'ence', 'able', 'ible', 'ment', 'ant', 'ent', 'ism',
                'ate', 'iti', 'ous', 'ive', 'ize']

    for suffix in suffixes:
        if token.endswith(suffix):
            token = token[:-len(suffix)]
            break # TODO: why are we breaking here? (there are better ways of doing this)

    return token

# test this function for "capital"
print(simple_stemmer(train_data.tokens))

In [ ]:
print("\nTest sample:")
print(test_data.head())

# Check data types of 'tokens' and 'ner_tags' columns
print("\nData type of train_data.tokens:")
print(type(train_data['tokens']))
print("\nData type of train_data.ner_tags:")
print(type(train_data['ner_tags']))

# Convert to list
train_tokens = train_data.tokens.tolist()
train_tags = train_data.ner_tags.tolist()
print("\nSample of training tokens:")
print(train_tokens[:20])
print("\nSample of training tags:")
print(train_tags[:5])

# fyi - View all tags
train_unique_tags = set(tag for sublist in train_tags for tag in sublist)
print("\nAll unique training NER tags:")
print(sorted(train_unique_tags))

todo

lexical stage
Morphological analysis procedures
include suf fix normalization, stemming and root
word lookup and analysis of internal p,mc~itation.

text-processing steps such as
 POS tagging, morphological analysis, and gazetteer
 matching.

## Byte pair encoding (BPE)

In [ ]:
import re
import collections

def compute_symbol_pairs_frequency(vocab):
    #Compute the frequency of adjacent symbol pairs in the vocabulary.

    symbol_pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            symbol_pairs[symbols[i], symbols[i+1]] += freq
    return symbol_pairs

def merge_vocabulary(symbol_pair, vocab_in):
    #Merge the given symbol pair in the vocabulary.

    vocab_out = {}
    bigram = re.escape(' '.join(symbol_pair))
    pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in vocab_in:
        word_out = pattern.sub(''.join(symbol_pair), word)
        vocab_out[word_out] = vocab_in[word]
    return vocab_out

def extract_symbol_pairs(word):
    #Return a set of symbol pairs in a word.

    symbol_pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        symbol_pairs.add((prev_char, char))
        prev_char = char
    return symbol_pairs

train_data = {'n a t u r a l </w>': 5, 'n a t u r e </w>': 2, 'l a n g u a g e </w>': 6, 'l a n g u a g e s </w>': 3}

bpe_codes = {}
bpe_codes_reverse = {}

num_merges = 10

for i in range(num_merges):
    pairs = compute_symbol_pairs_frequency(train_data)
    best_pair = max(pairs, key=pairs.get)
    train_data = merge_vocabulary(best_pair, train_data)

    bpe_codes[best_pair] = i
    bpe_codes_reverse[best_pair[0] + best_pair[1]] = best_pair

    print("new merge: {}".format(best_pair))
    print("train data: {}".format(train_data))

## tutorial

In [ ]:
import spacy

In [ ]:
nlp = spacy.blank("en")

doc = nlp('Dr. Strange loves pav bhaji of mumbai as it costs only £2 per plate')

for token in doc:
  print(token)

In [ ]:
doc[0]

In [ ]:
span = doc[1:5]
print(span)

In [ ]:
token0 = doc[0]
token0

In [ ]:
type(token0)

In [ ]:
dir(token0)

In [ ]:
token0.like_num

In [ ]:
token0.is_alpha

## huggingface tutorial

In [ ]:
#from transformers import pipeline

!pip install datasets

from datasets import load_dataset

dataset = load_dataset("nyu-mll/glue", "mrpc", split="train")

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def encode(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")

dataset = dataset.map(encode, batched=True)
dataset[0]

dataset = dataset.map(lambda examples: {"labels": examples["label"]}, batched=True)

import torch

dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

## NLP pipeline with huggingface dataset tutorial

## load dataset

In [ ]:
!pip install -U huggingface_hub
from huggingface_hub import login
login()
from datasets import load_dataset

dataset = load_dataset("parsa-mhmdi/Medical_NER")

ds_keys = dataset.keys()
print(ds_keys)

ds_size = dataset['train'].dataset_size
print(ds_size)

ds_desc = dataset['train'].description
print(ds_desc)

ds_features = dataset['train'].features
print(ds_features)

ds_sample = dataset['train'][0:1]
print(ds_sample)

dataset['train'].train_test_split(test_size=0.1)